In [3]:
import requests
import pandas as pd
import csv
from tqdm.notebook import tqdm, trange

In [4]:
# Get events from 01/01/2012 to 31/12/2021
date_range = ["2012-01-01", "2021-12-31"]
api_url = "https://mapacultural.secult.ce.gov.br/api"

# Occurrences path and params
occurrences_path = "/event/findOccurrences"
occurrences_params = {
    '@from': date_range[0],
    '@to': date_range[1],
}

# Event path and params
event_path = "/event/findOne"
def get_event_params(id: int) -> dict:
    return {
        'id': f'EQ({id})',
        '@select': 'id,singleUrl,name,subTitle,shortDescription,terms,classificacaoEtaria,registrationInfo',
        '@files': '(avatar.avatarBig):url'
    }
# Necessary requests headers
headers = {
    'User-Agent': 'Mozilla/5.0'
}

# 'Free' event keywords found in the dataset
free_keywords = [
    "", "0", "gratuito", "entrada franca",
    "entrada gratuita", "entrada livre", "livre",
    "r$ 00,00", "0,00", "grátis", "gratis", "free"
]

def replace_prices(s):
    if s.name.startswith('price'):
        return s.apply(lambda x: "Gratuito" if str(x).lower() in free_keywords else str(x))
    return s

# Function to clean df cols
def clean_cols(s):
    # Flatten lists
    if s.name.startswith('terms'):
        return s.apply(lambda x: ','.join(map(str, x)))
    # Normalize price
    replace_prices(s)

In [3]:
# Get occurrences and convert to JSON
print("Getting occurences from %s to %s, this may take a while." % (date_range[0], date_range[1]))

resp = requests.get(url=api_url+occurrences_path, params=occurrences_params, headers=headers)
occurrences = resp.json()
df = pd.DataFrame()

# Loop over occurrences (with tqdm for progress)
print("Getting events for each occurrence.")
for occurrence in tqdm(occurrences):
    # Request event with id
    res = requests.get(url=api_url+event_path, params=get_event_params(occurrence["id"]), headers=headers)
    # Set event meta from occurrence
    event_meta = {
        "id": occurrence["event_id"],
        "startsOn": occurrence["rule"]["startsOn"],
        "until": occurrence["rule"]["until"],
        "price": occurrence["rule"]["price"],
        "spaceId": occurrence["space"]["id"]
    }
    e = res.json()
    # Merge event data
    event = {**event_meta, **e}
    # Concat event to DataFrame
    df = pd.concat([df, pd.json_normalize(event)])

df

Getting occurences from 2012-01-01 to 2021-12-31, this may take a while.
Getting events for each occurrence.


  0%|          | 0/12859 [00:00<?, ?it/s]

,id,startsOn,until,price,spaceId,name,shortDescription,classificacaoEtaria,singleUrl,subTitle,registrationInfo,avatar,terms.tag,terms.linguagem
0,4394,2012-01-10,,Gratuito,1671,TWINDOW,“ Twindow” é uma tentativa de comunicação text...,Livre,https://mapacultural.secult.ce.gov.br/evento/4...,None,None,https://mapacultural.secult.ce.gov.br/files/ev...,,Artes Integradas
0,2266,2012-01-28,2012-02-11,"5,00",1304,Reino Folia - Pré-Carnaval Álvaro Weyne 2012,"O Bloco Reino Folia, nasceu, assim como outros...",16 anos,https://mapacultural.secult.ce.gov.br/evento/2...,Bloco de Carnaval de Rua do Álvaro Weyne,O evento contou com o apoio da Prefeitura de F...,https://mapacultural.secult.ce.gov.br/files/ev...,,Cultura Tradicional
0,2266,2012-01-28,2012-02-11,"5,00",1304,Reino Folia - Pré-Carnaval Álvaro Weyne 2012,"O Bloco Reino Folia, nasceu, assim como outros...",16 anos,https://mapacultural.secult.ce.gov.br/evento/2...,Bloco de Carnaval de Rua do Álvaro Weyne,O evento contou com o apoio da Prefeitura de F...,https://mapacultural.secult.ce.gov.br/files/ev...,,Cultura Tradicional
0,2266,2012-01-28,2012-02-11,"5,00",1304,Reino Folia - Pré-Carnaval Álvaro Weyne 2012,"O Bloco Reino Folia, nasceu, assim como outros...",16 anos,https://mapacultural.secult.ce.gov.br/evento/2...,Bloco de Carnaval de Rua do Álvaro Weyne,O evento contou com o apoio da Prefeitura de F...,https://mapacultural.secult.ce.gov.br/files/ev...,,Cultura Tradicional
0,6157,2012-04-19,,Gratuito,2979,5o. Abril pra Angola : no caminho da ancestral...,Evento tradicional realizado pelo centro cult...,Livre,https://mapacultural.secult.ce.gov.br/evento/6...,None,None,NaN,,Artes Integradas
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,7046,2021-06-24,2021-12-31,Gratuito,5391,A DESCOBERTA ALÉM DO ALCANCE DE GEORGIA GABRIE...,EXPERIMENTA!\n\nI MOSTRA DE EXPERIMENTOS CÊNIC...,Livre,https://mapacultural.secult.ce.gov.br/evento/7...,A descoberta além do alcance de Georgia Gabrie...,None,https://mapacultural.secult.ce.gov.br/files/ev...,,Teatro
0,7046,2021-06-24,2021-12-31,Gratuito,5391,A DESCOBERTA ALÉM DO ALCANCE DE GEORGIA GABRIE...,EXPERIMENTA!\n\nI MOSTRA DE EXPERIMENTOS CÊNIC...,Livre,https://mapacultural.secult.ce.gov.br/evento/7...,A descoberta além do alcance de Georgia Gabrie...,None,https://mapacultural.secult.ce.gov.br/files/ev...,,Teatro
0,7239,2021-10-13,2023-12-27,Gratuito,1636,Cururu Skate e Rap - A batalha.,Batalha de rap que ocorre todas as quartas fei...,Livre,https://mapacultural.secult.ce.gov.br/evento/7...,Batalha de mc's e evento cultural.,GRATUITA.,https://mapacultural.secult.ce.gov.br/files/ev...,,Hip Hop
0,7046,2021-06-24,2021-12-31,Gratuito,5391,A DESCOBERTA ALÉM DO ALCANCE DE GEORGIA GABRIE...,EXPERIMENTA!\n\nI MOSTRA DE EXPERIMENTOS CÊNIC...,Livre,https://mapacultural.secult.ce.gov.br/evento/7...,A descoberta além do alcance de Georgia Gabrie...,None,https://mapacultural.secult.ce.gov.br/files/ev...,,Teatro


In [5]:
# Load previous dataset
# df = pd.read_csv('mapa.csv', sep=';')
# df = df.apply(replace_prices)

# DataFrame cleanup
df = df.apply(clean_cols)
df = df.rename(columns={'@files:avatar.avatarBig.url': 'avatar'})
# Save to CSV
df.to_csv('mapa.csv', sep=';', index=None)

df

,id,startsOn,until,price,spaceId,name,shortDescription,classificacaoEtaria,singleUrl,subTitle,registrationInfo,avatar,terms.tag,terms.linguagem
0,4394,2012-01-10,NaN,Gratuito,1671,TWINDOW,“ Twindow” é uma tentativa de comunicação text...,Livre,https://mapacultural.secult.ce.gov.br/evento/4...,NaN,NaN,https://mapacultural.secult.ce.gov.br/files/ev...,NaN,Artes Integradas
1,2266,2012-01-28,2012-02-11,"5,00",1304,Reino Folia - Pré-Carnaval Álvaro Weyne 2012,"O Bloco Reino Folia, nasceu, assim como outros...",16 anos,https://mapacultural.secult.ce.gov.br/evento/2...,Bloco de Carnaval de Rua do Álvaro Weyne,O evento contou com o apoio da Prefeitura de F...,https://mapacultural.secult.ce.gov.br/files/ev...,NaN,Cultura Tradicional
2,2266,2012-01-28,2012-02-11,"5,00",1304,Reino Folia - Pré-Carnaval Álvaro Weyne 2012,"O Bloco Reino Folia, nasceu, assim como outros...",16 anos,https://mapacultural.secult.ce.gov.br/evento/2...,Bloco de Carnaval de Rua do Álvaro Weyne,O evento contou com o apoio da Prefeitura de F...,https://mapacultural.secult.ce.gov.br/files/ev...,NaN,Cultura Tradicional
3,2266,2012-01-28,2012-02-11,"5,00",1304,Reino Folia - Pré-Carnaval Álvaro Weyne 2012,"O Bloco Reino Folia, nasceu, assim como outros...",16 anos,https://mapacultural.secult.ce.gov.br/evento/2...,Bloco de Carnaval de Rua do Álvaro Weyne,O evento contou com o apoio da Prefeitura de F...,https://mapacultural.secult.ce.gov.br/files/ev...,NaN,Cultura Tradicional
4,6157,2012-04-19,NaN,Gratuito,2979,5o. Abril pra Angola : no caminho da ancestral...,Evento tradicional realizado pelo centro cult...,Livre,https://mapacultural.secult.ce.gov.br/evento/6...,NaN,NaN,NaN,NaN,Artes Integradas
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12854,7046,2021-06-24,2021-12-31,Gratuito,5391,A DESCOBERTA ALÉM DO ALCANCE DE GEORGIA GABRIE...,EXPERIMENTA!\n\nI MOSTRA DE EXPERIMENTOS CÊNIC...,Livre,https://mapacultural.secult.ce.gov.br/evento/7...,A descoberta além do alcance de Georgia Gabrie...,NaN,https://mapacultural.secult.ce.gov.br/files/ev...,NaN,Teatro
12855,7046,2021-06-24,2021-12-31,Gratuito,5391,A DESCOBERTA ALÉM DO ALCANCE DE GEORGIA GABRIE...,EXPERIMENTA!\n\nI MOSTRA DE EXPERIMENTOS CÊNIC...,Livre,https://mapacultural.secult.ce.gov.br/evento/7...,A descoberta além do alcance de Georgia Gabrie...,NaN,https://mapacultural.secult.ce.gov.br/files/ev...,NaN,Teatro
12856,7239,2021-10-13,2023-12-27,Gratuito,1636,Cururu Skate e Rap - A batalha.,Batalha de rap que ocorre todas as quartas fei...,Livre,https://mapacultural.secult.ce.gov.br/evento/7...,Batalha de mc's e evento cultural.,GRATUITA.,https://mapacultural.secult.ce.gov.br/files/ev...,NaN,Hip Hop
12857,7046,2021-06-24,2021-12-31,Gratuito,5391,A DESCOBERTA ALÉM DO ALCANCE DE GEORGIA GABRIE...,EXPERIMENTA!\n\nI MOSTRA DE EXPERIMENTOS CÊNIC...,Livre,https://mapacultural.secult.ce.gov.br/evento/7...,A descoberta além do alcance de Georgia Gabrie...,NaN,https://mapacultural.secult.ce.gov.br/files/ev...,NaN,Teatro
